# MongoDB Tutorial - Monday 22 February 2016

## Introduction

In what follows, we assume that you have installed MongoDB according to the [instructions online](https://docs.mongodb.org/manual/installation/) and started the mongodb daemon with the following command.

> mongod

Note that you might have to create a folder `/data/db` with appropriate access rights before the daemon starts successfully.

We also assume that you use Python 3 and have the [pymongo driver](http://api.mongodb.org/python/current/installation.html) installed and imported (see also **Setup** instructions at the end of the notebook).

** Note ** To run the notebook yourself, [install Jupyter](http://jupyter.readthedocs.org/en/latest/install.html), [download](https://raw.githubusercontent.com/mmathioudakis/moderndb/master/mongodb.tutorial.ipynb) the notebook, and [open it](http://jupyter.readthedocs.org/en/latest/running.html) with Jupyter.

** Note ** This notebook might be updated later. Major updates will be listed at the bottom of it, but you can also check the updates on [github](https://github.com/mmathioudakis/moderndb).

## "Hello World!" : Databases, Collections, Documents

*Relational databases* contain *tables* that contain *records*.

A **MongoDB database** contains **collections** that contain **documents**.

In [3]:
# start a client
client = pm.MongoClient() 

# connect to a database
db = client.moderndb

# get a collection
coll = db.test_collection

Documents follow the [JSON](http://json.org/) format and MongoDB stores them in a binary version of it ([BSON](http://bsonspec.org/)).
<img src = "http://json.org/object.gif">
<img src = "http://json.org/array.gif">
<img src = "http://json.org/value.gif">

Below you see examples of JSON documents.

** JSON example 0 **
```
{}
```

** JSON example 1 **
```
{
"name" : "Michael",
"age": 32,
"grades": [71, 85, 90, 34]
}
```

** JSON example 2 **

```
{
  "first name": "Michael",
  "last name": "Mathioudakis",
  "age": 32,
  "grades": {
    "ModernDB": 69,
    "Data Mining": 71,
    "Machine Learning": 95
  },
  "graduated": true,
  "previous schools": ["NTUA", "UofT"]
}
```

In Python, JSON documents are represented as dictionaries.
The examples from above are therefore represented as follows.

In [4]:
example_0 = {}

In [5]:
example_1 = {"name": "Michael", "age": 32, "grades": [71, 85, 90, 34]}

In [6]:
example_2 = \
{"first name": "Michael",
  "last name": "Mathioudakis",
  "age": 32,
  "grades": {
    "ModernDB": 69,
    "Data Mining": 71,
    "Machine Learning": 95
  },
  "graduated": True,
  "previous schools": ["NTUA", "UofT"]
}

Note that we can also use native Python objects, like the `datetime` object below, to specify values.

In [7]:
import datetime
example_3 = {"name": "Modern Database Systems",
          "start": datetime.datetime(2016, 1, 12),
          "end": datetime.datetime(2016, 3, 26),
          "tags": ["rdbms", "mongodb", "spark"]}

### Inserting and finding documents

Our collection `coll` is currently empty. Let's add one document to it.

In [8]:
coll.insert_one(example_0)

If we call the collection's function `find()`, we get back a cursor.

In [9]:
coll.find()

We can use the cursor to iterate over all documents in the collection.

In [10]:
for doc in coll.find():
    print(doc)

{'_id': ObjectId('56d07e9e12b680613210553a')}


Notice that the empty document we inserted is not *really* empty, but associated with an "\_id" key, added by MongoDB.

Let's try another one.

In [11]:
coll.insert_one(example_1)

In [12]:
for doc in coll.find():
    print(doc)
    print()

{'_id': ObjectId('56d07e9e12b680613210553a')}

{'_id': ObjectId('56d07e9e12b680613210553b'), 'age': 32, 'name': 'Michael', 'grades': [71, 85, 90, 34]}



Notice how MongoDB added an "\_id" for the new document, as well.
Let's insert more documents.

In [13]:
coll.insert_many([example_2, example_3])

In [14]:
for doc in coll.find():
    print(doc)
    print()

{'_id': ObjectId('56d07e9e12b680613210553a')}

{'_id': ObjectId('56d07e9e12b680613210553b'), 'age': 32, 'name': 'Michael', 'grades': [71, 85, 90, 34]}

{'_id': ObjectId('56d07e9f12b680613210553c'), 'last name': 'Mathioudakis', 'age': 32, 'first name': 'Michael', 'grades': {'Machine Learning': 95, 'Data Mining': 71, 'ModernDB': 69}, 'graduated': True, 'previous schools': ['NTUA', 'UofT']}

{'start': datetime.datetime(2016, 1, 12, 0, 0), '_id': ObjectId('56d07e9f12b680613210553d'), 'name': 'Modern Database Systems', 'tags': ['rdbms', 'mongodb', 'spark'], 'end': datetime.datetime(2016, 3, 26, 0, 0)}



Notice how the document we insert do not follow a schema?

Let us now find documents that match a condition -- let's say we want to find documents that have a field "name" with value "Michael".

In [15]:
query_result = coll.find({"name": "Michael"})
for doc in query_result:
    print(doc)

{'_id': ObjectId('56d07e9e12b680613210553b'), 'age': 32, 'name': 'Michael', 'grades': [71, 85, 90, 34]}


#### Projecting fields

We can use **find()** not only to retrieve documents that match a condition, but also to project only those fields that we are interested in. 

For example, to suppress the "\_id" field from appearing in the results, we can provide a second argument to __find()__, as follows.

In [16]:
query_result = coll.find({"name": "Michael"}, {"_id": 0})
for doc in query_result:
    print(doc)

{'age': 32, 'name': 'Michael', 'grades': [71, 85, 90, 34]}


What if we're interested in keeping only some of the rest of the fields -- let's say, only "grades"?

In [17]:
query_result = coll.find({"name": "Michael"}, {"_id": 0, "grades": 1})
for doc in query_result:
    print(doc)

{'grades': [71, 85, 90, 34]}


## Loading a larger dataset

Download file [dataset.json](https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/dataset.json) and load it into mongodb by running the command below.

In [19]:
%%bash
mongoimport --db moderndb --collection restaurants --drop --file dataset.json

2016-02-26T18:35:15.679+0200	connected to: localhost
2016-02-26T18:35:15.680+0200	dropping: moderndb.restaurants
2016-02-26T18:35:16.871+0200	imported 25359 documents


Alternatively, you can import the dataset by running the same command on a terminal.
> mongoimport --db moderndb --collection restaurants --drop --file dataset.json

The dataset contains documents that look like the one below.

** Restaurant Example **

```
{
  "address": {
     "building": "1007",
     "coord": [ -73.856077, 40.848447 ],
     "street": "Morris Park Ave",
     "zipcode": "10462"
  },
  "borough": "Bronx",
  "cuisine": "Bakery",
  "grades": [
     { "date": { "$date": 1393804800000 }, "grade": "A", "score": 2 },
     { "date": { "$date": 1378857600000 }, "grade": "A", "score": 6 },
     { "date": { "$date": 1358985600000 }, "grade": "A", "score": 10 },
     { "date": { "$date": 1322006400000 }, "grade": "A", "score": 9 },
     { "date": { "$date": 1299715200000 }, "grade": "B", "score": 14 }
  ],
  "name": "Morris Park Bake Shop",
  "restaurant_id": "30075445"
}
```

In [20]:
restaurants = db.restaurants # our new collection

In [21]:
# how many restaurants?
restaurants.count()

25359

## Querying the Dataset

In [22]:
# retrieve a cursor over all documents in the collection
cursor = restaurants.find()

In [23]:
# define printing function
def print_my_docs(cursor, num):
    for i in range(num): # print only up to num next documents from cursor
        try:
            print(next(cursor))
            print()
        except:
            break

In [24]:
# let's print a few documents
print_my_docs(cursor, 3)

{'_id': ObjectId('56d07ec4bda572c67c0a1b60'), 'name': "Wendy'S", 'grades': [{'score': 8, 'grade': 'A', 'date': datetime.datetime(2014, 12, 30, 0, 0)}, {'score': 23, 'grade': 'B', 'date': datetime.datetime(2014, 7, 1, 0, 0)}, {'score': 12, 'grade': 'A', 'date': datetime.datetime(2013, 4, 30, 0, 0)}, {'score': 12, 'grade': 'A', 'date': datetime.datetime(2012, 5, 8, 0, 0)}], 'restaurant_id': '30112340', 'address': {'coord': [-73.961704, 40.662942], 'street': 'Flatbush Avenue', 'zipcode': '11225', 'building': '469'}, 'cuisine': 'Hamburgers', 'borough': 'Brooklyn'}

{'_id': ObjectId('56d07ec4bda572c67c0a1b61'), 'name': 'Morris Park Bake Shop', 'grades': [{'score': 2, 'grade': 'A', 'date': datetime.datetime(2014, 3, 3, 0, 0)}, {'score': 6, 'grade': 'A', 'date': datetime.datetime(2013, 9, 11, 0, 0)}, {'score': 10, 'grade': 'A', 'date': datetime.datetime(2013, 1, 24, 0, 0)}, {'score': 9, 'grade': 'A', 'date': datetime.datetime(2011, 11, 23, 0, 0)}, {'score': 14, 'grade': 'B', 'date': datetime.

In [25]:
next(cursor) # get one more document

{'_id': ObjectId('56d07ec4bda572c67c0a1b63'),
 'address': {'building': '97-22',
  'coord': [-73.8601152, 40.7311739],
  'street': '63 Road',
  'zipcode': '11374'},
 'borough': 'Queens',
 'cuisine': 'Jewish/Kosher',
 'grades': [{'date': datetime.datetime(2014, 11, 24, 0, 0),
   'grade': 'Z',
   'score': 20},
  {'date': datetime.datetime(2013, 1, 17, 0, 0), 'grade': 'A', 'score': 13},
  {'date': datetime.datetime(2012, 8, 2, 0, 0), 'grade': 'A', 'score': 13},
  {'date': datetime.datetime(2011, 12, 15, 0, 0), 'grade': 'B', 'score': 25}],
 'name': 'Tov Kosher Kitchen',
 'restaurant_id': '40356068'}

### Specify equality conditions

In [26]:
# top-level field
cursor = restaurants.find({"borough": "Manhattan"})

print_my_docs(cursor, 2)

{'_id': ObjectId('56d07ec4bda572c67c0a1b6b'), 'name': '1 East 66Th Street Kitchen', 'grades': [{'score': 3, 'grade': 'A', 'date': datetime.datetime(2014, 5, 7, 0, 0)}, {'score': 4, 'grade': 'A', 'date': datetime.datetime(2013, 5, 3, 0, 0)}, {'score': 6, 'grade': 'A', 'date': datetime.datetime(2012, 4, 30, 0, 0)}, {'score': 0, 'grade': 'A', 'date': datetime.datetime(2011, 12, 27, 0, 0)}], 'restaurant_id': '40359480', 'address': {'coord': [-73.96926909999999, 40.7685235], 'street': 'East   66 Street', 'zipcode': '10065', 'building': '1'}, 'cuisine': 'American ', 'borough': 'Manhattan'}

{'_id': ObjectId('56d07ec4bda572c67c0a1b70'), 'name': 'Glorious Food', 'grades': [{'score': 12, 'grade': 'A', 'date': datetime.datetime(2014, 9, 2, 0, 0)}, {'score': 16, 'grade': 'B', 'date': datetime.datetime(2013, 12, 19, 0, 0)}, {'score': 9, 'grade': 'A', 'date': datetime.datetime(2013, 5, 28, 0, 0)}, {'score': 13, 'grade': 'A', 'date': datetime.datetime(2012, 12, 7, 0, 0)}, {'score': 11, 'grade': 'A',

In [27]:
# nested field (in embedded document)
cursor = restaurants.find({"address.zipcode": "10075"})

print_my_docs(cursor, 2)

{'_id': ObjectId('56d07ec4bda572c67c0a1c77'), 'name': 'Viand Cafe', 'grades': [{'score': 10, 'grade': 'A', 'date': datetime.datetime(2014, 5, 2, 0, 0)}, {'score': 15, 'grade': 'B', 'date': datetime.datetime(2013, 10, 21, 0, 0)}, {'score': 0, 'grade': 'B', 'date': datetime.datetime(2013, 4, 19, 0, 0)}, {'score': 40, 'grade': 'C', 'date': datetime.datetime(2012, 11, 21, 0, 0)}, {'score': 17, 'grade': 'B', 'date': datetime.datetime(2012, 4, 9, 0, 0)}], 'restaurant_id': '40369753', 'address': {'coord': [-73.9623333, 40.7757194], 'street': 'Madison Avenue', 'zipcode': '10075', 'building': '1011'}, 'cuisine': 'American ', 'borough': 'Manhattan'}

{'_id': ObjectId('56d07ec4bda572c67c0a1c78'), 'name': 'Nectar Coffee Shop', 'grades': [{'score': 5, 'grade': 'A', 'date': datetime.datetime(2014, 2, 20, 0, 0)}, {'score': 7, 'grade': 'A', 'date': datetime.datetime(2013, 8, 15, 0, 0)}, {'score': 11, 'grade': 'A', 'date': datetime.datetime(2013, 3, 8, 0, 0)}, {'score': 16, 'grade': 'B', 'date': dateti

In [28]:
# query by field in array
cursor = restaurants.find({"grades.grade": "B"})

In [29]:
# print one document from the query result
next(cursor)['grades']

[{'date': datetime.datetime(2014, 12, 30, 0, 0), 'grade': 'A', 'score': 8},
 {'date': datetime.datetime(2014, 7, 1, 0, 0), 'grade': 'B', 'score': 23},
 {'date': datetime.datetime(2013, 4, 30, 0, 0), 'grade': 'A', 'score': 12},
 {'date': datetime.datetime(2012, 5, 8, 0, 0), 'grade': 'A', 'score': 12}]

In [30]:
# exact array match
cursor = restaurants.find({"address.coord": [-73.98513559999999, 40.7676919]})

In [31]:
print_my_docs(cursor, 10)

{'_id': ObjectId('56d07ec4bda572c67c0a1ba1'), 'name': 'Dj Reynolds Pub And Restaurant', 'grades': [{'score': 2, 'grade': 'A', 'date': datetime.datetime(2014, 9, 6, 0, 0)}, {'score': 11, 'grade': 'A', 'date': datetime.datetime(2013, 7, 22, 0, 0)}, {'score': 12, 'grade': 'A', 'date': datetime.datetime(2012, 7, 31, 0, 0)}, {'score': 12, 'grade': 'A', 'date': datetime.datetime(2011, 12, 29, 0, 0)}], 'restaurant_id': '30191841', 'address': {'coord': [-73.98513559999999, 40.7676919], 'street': 'West   57 Street', 'zipcode': '10019', 'building': '351'}, 'cuisine': 'Irish', 'borough': 'Manhattan'}



### Specify Range Conditions

In [32]:
cursor = restaurants.find({"grades.score": {"$gt": 30}})

In [33]:
cursor = restaurants.find({"grades.score": {"$lt": 10}})

In [34]:
next(cursor)["grades"]

[{'date': datetime.datetime(2014, 12, 30, 0, 0), 'grade': 'A', 'score': 8},
 {'date': datetime.datetime(2014, 7, 1, 0, 0), 'grade': 'B', 'score': 23},
 {'date': datetime.datetime(2013, 4, 30, 0, 0), 'grade': 'A', 'score': 12},
 {'date': datetime.datetime(2012, 5, 8, 0, 0), 'grade': 'A', 'score': 12}]

### Multiple Conditions

In [35]:
# logical AND
cursor = restaurants.find({"cuisine": "Italian", "address.zipcode": "10075"})

In [36]:
next(cursor)

{'_id': ObjectId('56d07ec4bda572c67c0a1cba'),
 'address': {'building': '1133',
  'coord': [-73.958885, 40.7745559],
  'street': 'Lexington Avenue',
  'zipcode': '10075'},
 'borough': 'Manhattan',
 'cuisine': 'Italian',
 'grades': [{'date': datetime.datetime(2014, 8, 11, 0, 0),
   'grade': 'A',
   'score': 11},
  {'date': datetime.datetime(2013, 12, 10, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2013, 6, 10, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2012, 6, 8, 0, 0), 'grade': 'A', 'score': 13},
  {'date': datetime.datetime(2012, 1, 25, 0, 0), 'grade': 'A', 'score': 8},
  {'date': datetime.datetime(2011, 9, 13, 0, 0), 'grade': 'A', 'score': 12}],
 'name': 'Don Filippo Restaurant',
 'restaurant_id': '40372417'}

In [37]:
# logical OR
cursor = restaurants.find({"$or": [{"cuisine": "Italian"},
                                          {"address.zipcode": "10075"}]})

In [38]:
print_my_docs(cursor, 3)

{'_id': ObjectId('56d07ec4bda572c67c0a1b96'), 'name': 'Philadelhia Grille Express', 'grades': [{'score': 12, 'grade': 'A', 'date': datetime.datetime(2014, 2, 25, 0, 0)}, {'score': 7, 'grade': 'A', 'date': datetime.datetime(2013, 6, 27, 0, 0)}, {'score': 10, 'grade': 'A', 'date': datetime.datetime(2012, 12, 3, 0, 0)}, {'score': 12, 'grade': 'A', 'date': datetime.datetime(2011, 11, 9, 0, 0)}], 'restaurant_id': '40364305', 'address': {'coord': [-74.03400479999999, 40.6127077], 'street': '4 Avenue', 'zipcode': '11209', 'building': '10004'}, 'cuisine': 'Italian', 'borough': 'Brooklyn'}

{'_id': ObjectId('56d07ec4bda572c67c0a1b9d'), 'name': 'Isle Of Capri Resturant', 'grades': [{'score': 13, 'grade': 'A', 'date': datetime.datetime(2014, 9, 16, 0, 0)}, {'score': 10, 'grade': 'A', 'date': datetime.datetime(2014, 2, 24, 0, 0)}, {'score': 10, 'grade': 'A', 'date': datetime.datetime(2013, 5, 3, 0, 0)}, {'score': 7, 'grade': 'A', 'date': datetime.datetime(2012, 8, 20, 0, 0)}, {'score': 9, 'grade':

In [39]:
# logical AND, differently
cursor = restaurants.find({"$and": [{"cuisine": "Italian"},
                                          {"address.zipcode": "10075"}]})

In [40]:
next(cursor)

{'_id': ObjectId('56d07ec4bda572c67c0a1cba'),
 'address': {'building': '1133',
  'coord': [-73.958885, 40.7745559],
  'street': 'Lexington Avenue',
  'zipcode': '10075'},
 'borough': 'Manhattan',
 'cuisine': 'Italian',
 'grades': [{'date': datetime.datetime(2014, 8, 11, 0, 0),
   'grade': 'A',
   'score': 11},
  {'date': datetime.datetime(2013, 12, 10, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2013, 6, 10, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2012, 6, 8, 0, 0), 'grade': 'A', 'score': 13},
  {'date': datetime.datetime(2012, 1, 25, 0, 0), 'grade': 'A', 'score': 8},
  {'date': datetime.datetime(2011, 9, 13, 0, 0), 'grade': 'A', 'score': 12}],
 'name': 'Don Filippo Restaurant',
 'restaurant_id': '40372417'}

## Sorting

In [41]:
cursor = restaurants.find()

# to sort, specify list of sorting criteria,
# each criterion given as a tuple
# (field_name, sort_order)
# here we have only one
sorted_cursor = cursor.sort([("borough", pm.ASCENDING)]) 

In [42]:
print_my_docs(cursor, 2)

{'_id': ObjectId('56d07ec4bda572c67c0a1b61'), 'name': 'Morris Park Bake Shop', 'grades': [{'score': 2, 'grade': 'A', 'date': datetime.datetime(2014, 3, 3, 0, 0)}, {'score': 6, 'grade': 'A', 'date': datetime.datetime(2013, 9, 11, 0, 0)}, {'score': 10, 'grade': 'A', 'date': datetime.datetime(2013, 1, 24, 0, 0)}, {'score': 9, 'grade': 'A', 'date': datetime.datetime(2011, 11, 23, 0, 0)}, {'score': 14, 'grade': 'B', 'date': datetime.datetime(2011, 3, 10, 0, 0)}], 'restaurant_id': '30075445', 'address': {'coord': [-73.856077, 40.848447], 'street': 'Morris Park Ave', 'zipcode': '10462', 'building': '1007'}, 'cuisine': 'Bakery', 'borough': 'Bronx'}

{'_id': ObjectId('56d07ec4bda572c67c0a1b68'), 'name': 'Wild Asia', 'grades': [{'score': 11, 'grade': 'A', 'date': datetime.datetime(2014, 5, 28, 0, 0)}, {'score': 4, 'grade': 'A', 'date': datetime.datetime(2013, 6, 19, 0, 0)}, {'score': 3, 'grade': 'A', 'date': datetime.datetime(2012, 6, 15, 0, 0)}], 'restaurant_id': '40357217', 'address': {'coord'

In [43]:
another_sorted_cursor = restaurants.find().sort([("borough", pm.ASCENDING),
                                     ("address.zipcode", pm.DESCENDING)])

In [44]:
print_my_docs(another_sorted_cursor, 3)

{'_id': ObjectId('56d07ec4bda572c67c0a2990'), 'name': 'Carvel', 'grades': [{'score': 4, 'grade': 'A', 'date': datetime.datetime(2014, 2, 20, 0, 0)}, {'score': 6, 'grade': 'A', 'date': datetime.datetime(2013, 2, 4, 0, 0)}, {'score': 9, 'grade': 'A', 'date': datetime.datetime(2012, 1, 25, 0, 0)}, {'score': 12, 'grade': 'A', 'date': datetime.datetime(2011, 8, 17, 0, 0)}, {'score': 3, 'grade': 'P', 'date': datetime.datetime(2011, 5, 26, 0, 0)}], 'restaurant_id': '40876932', 'address': {'coord': [-73.89208789999999, 40.760344], 'street': '31 Avenue', 'zipcode': '11370', 'building': '7565'}, 'cuisine': 'Ice Cream, Gelato, Yogurt, Ices', 'borough': 'Bronx'}

{'_id': ObjectId('56d07ec4bda572c67c0a56b5'), 'name': "Antonio'S", 'grades': [{'score': 32, 'grade': 'C', 'date': datetime.datetime(2014, 10, 1, 0, 0)}, {'score': 18, 'grade': 'B', 'date': datetime.datetime(2014, 3, 13, 0, 0)}, {'score': 9, 'grade': 'A', 'date': datetime.datetime(2013, 5, 21, 0, 0)}, {'score': 12, 'grade': 'A', 'date': da

## Aggregation

Aggregation happens in stages.

In [45]:
# Group Documents by a Field and Calculate Count
cursor = restaurants.aggregate(
    [
        {"$group": {"_id": "$borough", "count": {"$sum": 1}}}
    ]
)

In [46]:
print_my_docs(cursor, 10)

{'_id': 'Missing', 'count': 51}

{'_id': 'Manhattan', 'count': 10259}

{'_id': 'Staten Island', 'count': 969}

{'_id': 'Bronx', 'count': 2338}

{'_id': 'Queens', 'count': 5656}

{'_id': 'Brooklyn', 'count': 6086}



In [47]:
# Filter and Group Documents
cursor = restaurants.aggregate(
    [
        {"$match": {"borough": "Queens", "cuisine": "Brazilian"}},
        {"$group": {"_id": "$address.zipcode", "count": {"$sum": 1}}}
    ]
)

In [48]:
print_my_docs(cursor, 10)

{'_id': '11377', 'count': 1}

{'_id': '11101', 'count': 2}

{'_id': '11368', 'count': 1}

{'_id': '11106', 'count': 3}

{'_id': '11103', 'count': 1}



In [49]:
# Filter and Group and then Filter Again documents
cursor = restaurants.aggregate(
    [
        {"$match": {"borough": "Manhattan", "cuisine": "American "}},
        {"$group": {"_id": "$address.zipcode", "count": {"$sum": 1}}},
        {"$match": {"count": {"$gt": 1}}}
    ]
)

In [50]:
print_my_docs(cursor, 10)

{'_id': '10463', 'count': 2}

{'_id': '10030', 'count': 7}

{'_id': '10281', 'count': 7}

{'_id': '10121', 'count': 3}

{'_id': '10040', 'count': 8}

{'_id': '10112', 'count': 7}

{'_id': '10044', 'count': 2}

{'_id': '10119', 'count': 4}

{'_id': '10031', 'count': 19}

{'_id': '10280', 'count': 6}



In [51]:
# Filter and Group and then Filter Again and then Sort Documents
cursor = restaurants.aggregate(
    [
        {"$match": {"borough": "Manhattan", "cuisine": "American "}},
        {"$group": {"_id": "$address.zipcode",  "count": {"$sum": 1}}},
        {"$match": {"count": {"$gt": 1}}},
        {"$sort":  {"count": -1, "_id": -1}}
    ]
)

In [52]:
print_my_docs(cursor, 10)

{'_id': '10019', 'count': 275}

{'_id': '10036', 'count': 270}

{'_id': '10003', 'count': 190}

{'_id': '10022', 'count': 188}

{'_id': '10001', 'count': 185}

{'_id': '10011', 'count': 153}

{'_id': '10014', 'count': 140}

{'_id': '10017', 'count': 133}

{'_id': '10012', 'count': 130}

{'_id': '10018', 'count': 129}



In [53]:
# Same but sort by multiple fields
# Filter and Group and then Filter Again and then Sort Documents
cursor = restaurants.aggregate(
    [
        {"$match": {"borough": "Manhattan", "cuisine": "American "}},
        {"$group": {"_id": "$address.zipcode",  "count": {"$sum": 1}}},
        {"$match": {"count": {"$gt": 1}}},
        {"$sort":  son.SON([("count", -1), ("_id", 1)])} # order matters!!
    ]
)

In [54]:
print_my_docs(cursor, 10)

{'_id': '10019', 'count': 275}

{'_id': '10036', 'count': 270}

{'_id': '10003', 'count': 190}

{'_id': '10022', 'count': 188}

{'_id': '10001', 'count': 185}

{'_id': '10011', 'count': 153}

{'_id': '10014', 'count': 140}

{'_id': '10017', 'count': 133}

{'_id': '10012', 'count': 130}

{'_id': '10018', 'count': 129}



In [55]:
# what will this do?
cursor = restaurants.aggregate(
    [
        {"$group": {"_id": None, "count": {"$sum": 1}} }
    ]
)

In [56]:
print_my_docs(cursor, 10)

{'_id': None, 'count': 25359}



In [57]:
# projection
# what will this do?
cursor = restaurants.aggregate(
    [
        {"$group": {"_id": "$address.zipcode", "count": {"$sum": 1}}},
        {"$project": {"_id": 0, "count": 1}}
    ]
)

In [58]:
print_my_docs(cursor, 10)

{'count': 1}

{'count': 1}

{'count': 1}

{'count': 1}

{'count': 2}

{'count': 1}

{'count': 1}

{'count': 2}

{'count': 1}

{'count': 1}



In [59]:
# what will this do?
cursor = restaurants.aggregate(
    [
        {"$group": {"_id": {"cuisine": "$cuisine"}, "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
)

In [60]:
print_my_docs(cursor, 5)

{'_id': {'cuisine': 'American '}, 'count': 6183}

{'_id': {'cuisine': 'Chinese'}, 'count': 2418}

{'_id': {'cuisine': 'Café/Coffee/Tea'}, 'count': 1214}

{'_id': {'cuisine': 'Pizza'}, 'count': 1163}

{'_id': {'cuisine': 'Italian'}, 'count': 1069}



In [61]:
# what will this do?
cursor = restaurants.aggregate(
    [
        {"$group": {"_id": {"zip": "$address.zipcode"}, "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
)

In [62]:
print_my_docs(cursor, 5)

{'_id': {'zip': '10003'}, 'count': 686}

{'_id': {'zip': '10019'}, 'count': 675}

{'_id': {'zip': '10036'}, 'count': 611}

{'_id': {'zip': '10001'}, 'count': 520}

{'_id': {'zip': '10022'}, 'count': 485}



In [63]:
# what will this do?
cursor = restaurants.aggregate(
    [
        {"$group": {"_id": {"cuisine": "$cuisine", "zip": "$address.zipcode"}, "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
)

In [64]:
print_my_docs(cursor, 5)

{'_id': {'zip': '10019', 'cuisine': 'American '}, 'count': 275}

{'_id': {'zip': '10036', 'cuisine': 'American '}, 'count': 270}

{'_id': {'zip': '10003', 'cuisine': 'American '}, 'count': 190}

{'_id': {'zip': '10022', 'cuisine': 'American '}, 'count': 188}

{'_id': {'zip': '10001', 'cuisine': 'American '}, 'count': 185}



### Limiting the number of results

In [65]:
# what will this do?

cursor = restaurants.aggregate(
    [
        {"$group": {"_id": {"cuisine": "$cuisine", "zip": "$address.zipcode"}, "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 10} # See comment under "In-class questions"
    ]
)

for doc in cursor:
    print(doc["_id"]["cuisine"], doc["_id"]["zip"], doc["count"])

American  10019 275
American  10036 270
American  10003 190
American  10022 188
American  10001 185
American  10011 153
American  10014 140
American  10017 133
American  10012 130
American  10018 130


### Storing the result as a collection

We can use operator [\$out](https://docs.mongodb.org/manual/reference/operator/aggregation/out/) in a final stage to store the result of a query into a new collection. The following example selects restaurants from Manhattan and stores them in their own collection in the same database.

In [66]:
restaurants.aggregate(
    [
        {"$match": {"borough": "Manhattan"}},
        {"$out": "manhattan"}
    ]
)

## SQL to Aggregation

Here we explore the correspondence between SQL queries and the aggregation framework.

** SQL query **
```
SELECT COUNT(*) AS count
FROM restaurants
```

In [67]:
cursor = restaurants.aggregate(
    [
        {"$group": {"_id": None, "count": {"$sum": 1}} }
    ]
)

** SQL query **
```
SELECT borough, cuisine, COUNT(*) as count
FROM restaurants
GROUP BY borough, cuisine
```

In [68]:
cursor = restaurants.aggregate(
    [
        {"$group": {"_id": {"borough": "$borough", "cuisine": "$cuisine"}, "count": {"$sum": 1}}}
    ]
)

** SQL query **
```
SELECT borough, cuisine, COUNT(*) as count
FROM restaurants
GROUP BY borough, cuisine
HAVING COUNT(*) > 3
```

In [69]:
cursor = restaurants.aggregate(
    [
        {"$group": {"_id": {"borough": "$borough", "cuisine": "$cuisine"}, "count": {"$sum": 1}}},
        {"$match": {"count": {"$gt": 3}}}
    ]
)

** SQL Query **
```
SELECT zipcode, cuisine, COUNT(*) as count
FROM restaurants
WHERE borough = "Manhattan"
GROUP BY zipcode, cuisine
HAVING COUNT(*) > 3
```

In [70]:
cursor = restaurants.aggregate(
    [
        {"$match": {"borough": "Manhattan"}},
        {"$group": {"_id": {"zipcode": "$address.zipcode", "cuisine": "$cuisine"}, "count": {"$sum": 1}}},
        {"$match": {"count": {"$gt": 3}}}
    ]
)

In [71]:
print_my_docs(cursor, 5)

{'_id': {'cuisine': 'Other', 'zipcode': '10035'}, 'count': 5}

{'_id': {'cuisine': 'Other', 'zipcode': '10023'}, 'count': 4}

{'_id': {'cuisine': 'Other', 'zipcode': '10018'}, 'count': 5}

{'_id': {'cuisine': 'Other', 'zipcode': '10004'}, 'count': 6}

{'_id': {'cuisine': 'Other', 'zipcode': '10022'}, 'count': 5}



** SQL Query **
```
SELECT zipcode, cuisine, COUNT(*) as count
FROM restaurants
WHERE borough = "Manhattan"
GROUP BY zipcode, cuisine
HAVING COUNT(*) > 3
ORDER BY count
```

In [72]:
cursor = restaurants.aggregate(
    [
        {"$match": {"borough": "Manhattan"}},
        {"$group": {"_id": {"zipcode": "$address.zipcode", "cuisine": "$cuisine"}, "count": {"$sum": 1}}},
        {"$match": {"count": {"$gt": 3}}},
        {"$sort": {"count": 1}}
    ]
)

## Using secondary memory (disk)

In [73]:
cursor = restaurants.aggregate(
    [
        {"$match": {"borough": "Manhattan"}},
        {"$group": {"_id": {"zipcode": "$address.zipcode", "cuisine": "$cuisine"}, "count": {"$sum": 1}}},
        {"$match": {"count": {"$gt": 3}}},
        {"$sort": {"count": 1}}
    ],
    allowDiskUse = True # this can be useful when data does not fit in memory, e.g., to perform external sorting
)

## Indexing

MongoDb automatically creates an index on the `_id` field upon creating a collection.
We can use `create_index()` to create index on one or more fields of a collection.

### Single-field index

In [74]:
# note that the argument is a list of tuples
# [(<field>: <type>), ...]
# here, we specify only one such tuple for one field
restaurants.create_index([("borough", pm.ASCENDING)])

'borough_1'

The index is created only if it does not already exist.

### Compound index

In [75]:
# compound index (more than one indexed fields)
restaurants.create_index([
    ("cuisine", pm.ASCENDING),
    ("address.zipcode", pm.DESCENDING)
])

'cuisine_1_address.zipcode_-1'

### Deleting indexes

In [76]:
restaurants.drop_index('borough_1') # drop this index

In [77]:
restaurants.drop_index('cuisine_1_address.zipcode_-1') # drop that index

In [78]:
restaurants.drop_indexes() # drop all indexes!!1

### Multi-key index

An index for a fields with array value.

In [79]:
restaurants.find_one()

{'_id': ObjectId('56d07ec4bda572c67c0a1b60'),
 'address': {'building': '469',
  'coord': [-73.961704, 40.662942],
  'street': 'Flatbush Avenue',
  'zipcode': '11225'},
 'borough': 'Brooklyn',
 'cuisine': 'Hamburgers',
 'grades': [{'date': datetime.datetime(2014, 12, 30, 0, 0),
   'grade': 'A',
   'score': 8},
  {'date': datetime.datetime(2014, 7, 1, 0, 0), 'grade': 'B', 'score': 23},
  {'date': datetime.datetime(2013, 4, 30, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2012, 5, 8, 0, 0), 'grade': 'A', 'score': 12}],
 'name': "Wendy'S",
 'restaurant_id': '30112340'}

In [80]:
restaurants.create_index([("address.coord", 1)])

'address.coord_1'

In [81]:
restaurants.create_index([("grades.score", 1)])

'grades.score_1'

In [82]:
restaurants.create_index([("grades.grade", 1), ("grades.score", 1)])

'grades.grade_1_grades.score_1'

The following will not work!
We cannot _currently_ have compound multi-key indexes.

In [83]:
restaurants.create_index([("address.coord", 1), ("grades.score", 1)]) # NOPE!

OperationFailure: command SON([('createIndexes', 'restaurants'), ('indexes', [{'name': 'address.coord_1_grades.score_1', 'key': SON([('address.coord', 1), ('grades.score', 1)])}])]) on namespace moderndb.$cmd failed: cannot index parallel arrays [grades] [coord]

## Retrieving the execution plan

We can retrieve the execution plan for a **find()** query by calling the [explain()](https://docs.mongodb.org/manual/reference/method/cursor.explain/) function on the result cursor. We demonstrate this in the following example.

In [84]:
restaurants.drop_indexes() # we drop all indexes first -- use this with care!
restaurants.create_index([("borough", pm.ASCENDING)]) # build an index on field "borough", in ascending order
my_cursor = restaurants.find({"borough": "brooklyn"}) # submit query to find restaurants from specific borough
my_cursor.explain()["queryPlanner"]["winningPlan"] # ask mongodb to explain execution plan

{'inputStage': {'direction': 'forward',
  'indexBounds': {'borough': ['["brooklyn", "brooklyn"]']},
  'indexName': 'borough_1',
  'indexVersion': 1,
  'isMultiKey': False,
  'isPartial': False,
  'isSparse': False,
  'isUnique': False,
  'keyPattern': {'borough': 1},
  'stage': 'IXSCAN'},
 'stage': 'FETCH'}

As we see in this example, MongoDB makes use of an index (as indicated by keyword "IXSCAN") -- and particularly the index ('borough_1') we constructed to execute the query.
What if we had not built this index?

In [85]:
restaurants.drop_indexes() # we drop all indexes first -- use this with care!
my_cursor = restaurants.find({"borough": "brooklyn"}) # submit query to find restaurants from specific borough
my_cursor.explain()["queryPlanner"]["winningPlan"] # ask mongodb to explain execution plan

{'direction': 'forward',
 'filter': {'borough': {'$eq': 'brooklyn'}},
 'stage': 'COLLSCAN'}

In that case, MongoDB simply performs a scan over the collection (as indicated by keyword "COLLSCAN").

## Joins

Until very recently, MongoDB did not support joins.
It was up to the user to implement a join if needed.

For example, the following double for-loop emulated "page-oriented nested-loops join".

In [86]:
for a in restaurants.find({"borough": "Manhattan"}).limit(7):
    for b in restaurants.find({"borough": "Bronx"}).limit(5):
        if a["cuisine"] == b["cuisine"]:
            print(a["cuisine"], a["address"]["zipcode"], b["address"]["zipcode"])
    

American  10065 10460
American  10021 10460
American  10025 10460
American  10012 10460


### Joins with \$lookup

This is a new aggregation stage that implements *left outer equi-joins*.

"A [left outer equi-join](https://www.mongodb.com/blog/post/joins-and-other-aggregation-enhancements-coming-in-mongodb-3-2-part-1-of-3-introduction) produces a result set that contains data for all documents from the left table (collection) together with data from the right table (collection) for documents where there is a match with documents from the left table (collection)."

In [87]:
# create first collection
orders_docs = [{ "_id" : 1, "item" : "abc", "price" : 12, "quantity" : 2 },
{ "_id" : 2, "item" : "jkl", "price" : 20, "quantity" : 1 },
{ "_id" : 3  }]
orders = db.orders
orders.drop()
orders.insert_many(orders_docs)

In [88]:
# create second collection
inventory_docs = [
    { "_id" : 1, "item" : "abc", "description": "product 1", "instock" : 120 },
{ "_id" : 2, "item" : "def", "description": "product 2", "instock" : 80 },
{ "_id" : 3, "item" : "ijk", "description": "product 3", "instock" : 60 },
{ "_id" : 4, "item" : "jkl", "description": "product 4", "instock" : 70 },
{ "_id" : 5, "item": None, "description": "Incomplete" },
{ "_id" : 6 }
]
inventory = db.inventory
inventory.drop()
inventory.insert_many(inventory_docs)

In [89]:
result = orders.aggregate([ # "orders" is the outer collection
    {
      "$lookup":
        {
          "from": "inventory", # the inner collection 
          "localField": "item", # the join field of the outer collection
          "foreignField": "item", # the join field of the outer collection
          "as": "inventory_docs" # name of field with array of joined inner docs
        }
   }
])

In [90]:
print_my_docs(result, 10)

{'item': 'abc', '_id': 1, 'inventory_docs': [{'item': 'abc', '_id': 1, 'instock': 120, 'description': 'product 1'}], 'price': 12, 'quantity': 2}

{'item': 'jkl', '_id': 2, 'inventory_docs': [{'item': 'jkl', '_id': 4, 'instock': 70, 'description': 'product 4'}], 'price': 20, 'quantity': 1}

{'inventory_docs': [{'item': None, '_id': 5, 'description': 'Incomplete'}, {'_id': 6}], '_id': 3}



## In-class questions

### Question: How do we query for documents with an array field, all the elements of which satisfy a condition?

Two approaches (if you can think of a different approach, please let me know):
* Use the [**\$not**](https://docs.mongodb.org/manual/reference/operator/query/not/#op._S_not) operators: form a query to express that "there is no element in the array that does not satisfy the condition".
* In aggregation, combine an [**\$unwind**](https://docs.mongodb.org/manual/reference/operator/aggregation/unwind/) stage with a [**$group**](https://docs.mongodb.org/manual/reference/operator/aggregation/group/) stage.

To provide an example, let's say we want to __find restaurants with 'A' grades only__.
Below we show how we can use each of the aforementioned approaches.

#### First approach: using $not

In [91]:
# using the $not operator
# "find restaurants that contain no grades that are not equal to A"
cursor = restaurants.find({"grades.grade": {"$exists": True}, "grades": {"$not": {"$elemMatch": {"grade": {"$ne": "A"}}}}})

In [92]:
print_my_docs(cursor, 3)

{'_id': ObjectId('56d07ec4bda572c67c0a1b64'), 'name': 'Kosher Island', 'grades': [{'score': 9, 'grade': 'A', 'date': datetime.datetime(2014, 10, 6, 0, 0)}, {'score': 12, 'grade': 'A', 'date': datetime.datetime(2014, 5, 20, 0, 0)}, {'score': 12, 'grade': 'A', 'date': datetime.datetime(2013, 4, 4, 0, 0)}, {'score': 9, 'grade': 'A', 'date': datetime.datetime(2012, 1, 24, 0, 0)}], 'restaurant_id': '40356442', 'address': {'coord': [-74.1377286, 40.6119572], 'street': 'Victory Boulevard', 'zipcode': '10314', 'building': '2206'}, 'cuisine': 'Jewish/Kosher', 'borough': 'Staten Island'}

{'_id': ObjectId('56d07ec4bda572c67c0a1b65'), 'name': "Wilken'S Fine Food", 'grades': [{'score': 10, 'grade': 'A', 'date': datetime.datetime(2014, 5, 29, 0, 0)}, {'score': 10, 'grade': 'A', 'date': datetime.datetime(2014, 1, 14, 0, 0)}, {'score': 8, 'grade': 'A', 'date': datetime.datetime(2013, 8, 3, 0, 0)}, {'score': 10, 'grade': 'A', 'date': datetime.datetime(2012, 7, 18, 0, 0)}, {'score': 13, 'grade': 'A', '

##### Note on the semantics of the \$not operator

The operator selects documents that _do not match_ the specified condition on the specified field. These documents include ones that _do not contain_ the field.

To demonstrate this, consider the following simple example of a collection.

In [93]:
# simple example of a collection
mycoll = db.mycoll
mycoll.drop()

# insert three documents
mycoll.insert_one({"grades": [7, 7]})
mycoll.insert_one({"grades": [7, 3]})
mycoll.insert_one({"grades": [3, 3]})
mycoll.insert_one({"grades": []})
mycoll.insert_one({})

The result of the following query contains documents that do not contain the "grades" field.

In [94]:
# find documents that have no "grades" elements that are not equal to "A"
mycursor = mycoll.find({"grades": {"$not": {"$elemMatch": {"$ne": 7}}}})
print_my_docs(mycursor, 10)

{'_id': ObjectId('56d07f1112b680613210553e'), 'grades': [7, 7]}

{'_id': ObjectId('56d07f1112b6806132105541'), 'grades': []}

{'_id': ObjectId('56d07f1112b6806132105542')}



We can remove such documents from the result as a post-processing step. (**Exercise**: how?)

#### Second approach: aggregation pipeline

In [95]:
# using aggregation
mycursor = restaurants.aggregate(
    [
        # unwind the grades array
        {"$unwind": "$grades"}, #now each document contains one "grades" value
        
        # group by document "_id" and count:
        # (i) the total number of documents in each group as `count`
        # -- this is the same as the number of elements in the original array
        # (ii) the number of documents that satisfy the condition (grade = "A") as `num_satisfied`
        {"$group": {"_id": "$_id", "count": {"$sum": 1}, "num_satisfied": {"$sum": {"$cond": [{"$eq": ["$grades.grade", "A"]}, 1, 0]}}}},
        
        # create a field (named `same`) that is 1 if (count = num_satisfied) and 0 otherwise
        {"$project": {"_id": 1, "same_count": {"$cond": [{"$eq": ["$count", "$num_satisfied"]} , 1, 0]}}},
        
        # keep only the document ids for which (same = 1)
        {"$match": {"same_count": 1}}
    ]
)

print_my_docs(mycursor, 5)

{'_id': ObjectId('56d07ec4bda572c67c0a7dfa'), 'same_count': 1}

{'_id': ObjectId('56d07ec4bda572c67c0a7dce'), 'same_count': 1}

{'_id': ObjectId('56d07ec4bda572c67c0a7dc4'), 'same_count': 1}

{'_id': ObjectId('56d07ec4bda572c67c0a7dae'), 'same_count': 1}

{'_id': ObjectId('56d07ec4bda572c67c0a7d77'), 'same_count': 1}



## Question: Does MongoDB optimize the stages of an aggregation pipeline?

The question was asked in relation to the "limit" query we saw above ("Limiting the number of results").

Indeed, MongoDB does optimize the execution of the aggregation pipeline, as explained [here](https://docs.mongodb.org/manual/core/aggregation-pipeline-optimization/). In relation to the aforementioned query, see, in particular, the part on [sort+limit coalescence](https://docs.mongodb.org/manual/core/aggregation-pipeline-optimization/#sort-limit-coalescence).

***

# Credits and references

We used and consulted material from:
* the offficial [PyMongo tutorial](https://docs.mongodb.org/getting-started/python/) as well as this shorter [one](http://api.mongodb.org/python/current/tutorial.html),
* the [JSON](http://json.org/) and [BSON](http://bsonspec.org) documentation, as well as [SON](http://api.mongodb.org/python/current/api/bson/son.html#bson.son.SON),
* these [posts](https://www.mongodb.com/blog/post/joins-and-other-aggregation-enhancements-coming-in-mongodb-3-2-part-1-of-3-introduction) on the MongoDB blog about the new (v.3.2) left outer equi-join functionality,
* this [StackOverflow thread](http://stackoverflow.com/questions/18123300/mongo-array-query-only-find-where-all-elements-match).

***
# Updates to the notebook
* Compared to the in-class tutorial, we've added answers to two in-class questions.
* [Feb 26] Added sub-sections **projecting fields**, **storing the result as a collection**, and ** retrieving the execution plan**.

***

# Setup

Run the following commands before the rest of the notebook.

In [1]:
import pymongo as pm
client = pm.MongoClient()
client.drop_database("moderndb")

In [2]:
import bson.son as son